#from all cools page: https://lhqing.github.io/ALLCools/cell_level/dmg/02-DifferentiallyMethylatedGenes.html



In [1]:
import pandas as pd
import anndata
from ALLCools.clustering import one_vs_rest_dmg
import xarray as xr
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

from ALLCools.mcds import MCDS
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['savefig.transparent'] = True
matplotlib.rcParams['path.simplify'] = False  # keep exact vectors

In [2]:
mcds_paths = './genebody_frac.mcds/'
cell_meta_path = 'L1_clustering_labels.csv'
cluster_col = 'L1'

obs_dim = 'cell'
var_dim = 'genebody'
mc_type = 'CGN'

top_n = 1000
auroc_cutoff = 0.5
adj_p_cutoff = 0.1
fc_cutoff = 0.8
max_cluster_cells = 2000
max_other_fold = 5

cpu = 20

In [3]:
adata_path = 'adata.with_coords_final_pooled.h5ad'
adata = anndata.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 46 × 23803
    obs: 'leiden', 'mCGFrac', 'TotalRead', 'Group', 'MT.Fraction', 'Collection', 'Year', 'Type', 'umap_0', 'umap_1', 'L1', 'L1_proba', 'AnimalID'
    var: 'chrom', 'end', 'start', 'CHN_mean', 'CHN_dispersion', 'CHN_cov', 'CHN_score', 'CHN_feature_select', 'CGN_mean', 'CGN_dispersion', 'CGN_cov', 'CGN_score', 'CGN_feature_select', 'leiden_enriched_features'
    uns: 'AnimalID_colors', 'Collection_colors', 'Group_colors', 'L1_colors', 'Type_colors', 'Year_colors', 'leiden', 'leiden_colors', 'leiden_feature_enrichment', 'leiden_sizes', 'log', 'neighbors', 'paga', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    obsp: 'connectivities', 'distances'

In [4]:
cell_metaa = adata.obs
#cell_meta.to_csv("L1_clustering_labels_expanded.csv")
cell_metaa

,leiden,mCGFrac,TotalRead,Group,MT.Fraction,Collection,Year,Type,umap_0,umap_1,L1,L1_proba,AnimalID
cell,,,,,,,,,,,,,
4146045001PO3NC4G4C,0,0.689,2200000,Control,NaN,2,2023,PO,NaN,-2.157124,c0,1,41460
4129944963PO2NC4M1S,0,0.684,2300000,Stressed,NaN,1,2023,PO,NaN,0.059123,c0,1,41299
4520844650PO1NC5A7S,2,0.676,2400000,Stressed,NaN,1,2022,PO,NaN,-1.432278,c0,1,45208
4146045001PO2NC3F4C,1,0.686,2500000,Control,NaN,2,2023,PO,NaN,5.192831,c0,1,41460
4520844650PO2NC3B7S,2,0.680,2600000,Stressed,NaN,1,2022,PO,NaN,-2.382400,c0,1,45208
3905345005PO1NC2I1S,1,0.623,2700000,Stressed,NaN,1,2023,PO,NaN,3.336180,c0,1,39053
4129944963PO1NC3N1S,0,0.626,2900000,Stressed,NaN,2,2023,PO,NaN,2.971206,c0,1,41299
4211144957PO1NC3K4C,2,0.674,3200000,Control,NaN,1,2023,PO,NaN,0.912201,c0,1,42111
4660644938PO2NC3A10C,1,0.680,3600000,Control,NaN,1,2023,PO,NaN,4.396003,c0,1,46606


In [5]:
gene_meta = pd.read_csv('../../GeneMetadata_withGeneName_corrected.csv')
gene_meta["Gene"] = gene_meta["gene_id"]
gene_meta

,gene_id,chrom,start,end,gene_name,Gene
0,ENSMMUG00000023296,chr1,8230,26653,NaN,ENSMMUG00000023296
1,ENSMMUG00000000634,chr1,71581,83178,ZNF692,ENSMMUG00000000634
2,ENSMMUG00000037875,chr1,81257,91679,ZNF672,ENSMMUG00000037875
3,ENSMMUG00000000632,chr1,102661,117835,SH3BP5L,ENSMMUG00000000632
4,ENSMMUG00000064799,chr1,122995,124202,NaN,ENSMMUG00000064799
...,...,...,...,...,...,...
27133,ENSMMUG00000052253,chrY,10095285,10099690,NaN,ENSMMUG00000052253
27134,ENSMMUG00000057875,chrY,10102204,10117522,NaN,ENSMMUG00000057875
27135,ENSMMUG00000063386,chrY,10327102,10371239,NaN,ENSMMUG00000063386
27136,ENSMMUG00000054981,chrY,10401312,10421795,NaN,ENSMMUG00000054981


In [6]:



def calculate_dmgs_scanpy(adata, cluster_col='L1', group_col='Group', method='wilcoxon', 
                          top_n=1000, adj_p_cutoff=0.1, pval_cutoff=0.05, logfc_min=0.8):
    # Store results
    dmg_results = []
    
    # Iterate over each cluster
    clusters = adata.obs[cluster_col].unique()
    for cluster in clusters:
        # Subset data for the current cluster
        adata_cluster = adata[adata.obs[cluster_col] == cluster]
        
        # Perform differential analysis between Control and Stress within the cluster
        sc.tl.rank_genes_groups(adata_cluster, groupby=group_col, method=method)
        
        # Extract results for the comparison (e.g., Control vs. Stress)
        rank_genes_groups = adata_cluster.uns['rank_genes_groups']
        genes = rank_genes_groups['names']['Stressed']
        pvals = rank_genes_groups['pvals']['Stressed']
        adjpvals = rank_genes_groups['pvals_adj']['Stressed']
        logfc = rank_genes_groups['logfoldchanges']['Stressed']
        
        # Collect DMGs based on adjusted p-value and log fold change criteria
        # Collect DMGs based on adjusted p-value and log fold change criteria
        dmg_cluster = []
        for gene, pval, adjpval, lfc in zip(genes, pvals, adjpvals, logfc):
            if pval < pval_cutoff and abs(lfc) > logfc_min:
                direction = 'hyper methylation' if lfc > 0 else 'hypo methylation'
                dmg_cluster.append({
                    'Cluster': cluster,
                    'Gene': gene,
                    'pval': pval,
                    'adjpval': adjpval,
                    'logfc': lfc,
                    'Direction': direction
                })    
        # Sort by adjusted p-value and limit to top_n
        dmg_cluster = sorted(dmg_cluster, key=lambda x: x['pval'])[:top_n]
        dmg_results.extend(dmg_cluster)
    
    # Convert to DataFrame
    dmg_table = pd.DataFrame(dmg_results)
    return dmg_table




In [7]:
mcds = MCDS.open(mcds_paths)
mcds

<xarray.MCDS>
Dimensions:            (cell: 46, genebody: 26940, mc_type: 2)
Coordinates:
  * cell               (cell) <U20 '4146045001PO3NC4G4C' ... '4123744641PO1NC...
  * genebody           (genebody) <U18 'ENSMMUG00000023296' ... 'ENSMMUG00000...
    genebody_chrom     (genebody) <U5 'chr1' 'chr1' 'chr1' ... 'chrX' 'chrX'
    genebody_cov_mean  (genebody) float64 964.3 1.281e+03 ... 261.2 69.3
    genebody_end       (genebody) int64 26653 83178 ... 151987003 152565575
    genebody_start     (genebody) int64 8230 71581 81257 ... 151983199 152564067
  * mc_type            (mc_type) <U3 'CGN' 'CHN'
Data variables:
    genebody_da_frac   (cell, genebody, mc_type) float32 dask.array<chunksize=(46, 26940, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  genebody

In [8]:



    
    # Filter metadata to ensure cells align with MCDS cells
cell_meta = cell_metaa[cell_metaa.index.isin(mcds['cell'].values)]
cell_meta

,leiden,mCGFrac,TotalRead,Group,MT.Fraction,Collection,Year,Type,umap_0,umap_1,L1,L1_proba,AnimalID
cell,,,,,,,,,,,,,
4146045001PO3NC4G4C,0,0.689,2200000,Control,NaN,2,2023,PO,NaN,-2.157124,c0,1,41460
4129944963PO2NC4M1S,0,0.684,2300000,Stressed,NaN,1,2023,PO,NaN,0.059123,c0,1,41299
4520844650PO1NC5A7S,2,0.676,2400000,Stressed,NaN,1,2022,PO,NaN,-1.432278,c0,1,45208
4146045001PO2NC3F4C,1,0.686,2500000,Control,NaN,2,2023,PO,NaN,5.192831,c0,1,41460
4520844650PO2NC3B7S,2,0.680,2600000,Stressed,NaN,1,2022,PO,NaN,-2.382400,c0,1,45208
3905345005PO1NC2I1S,1,0.623,2700000,Stressed,NaN,1,2023,PO,NaN,3.336180,c0,1,39053
4129944963PO1NC3N1S,0,0.626,2900000,Stressed,NaN,2,2023,PO,NaN,2.971206,c0,1,41299
4211144957PO1NC3K4C,2,0.674,3200000,Control,NaN,1,2023,PO,NaN,0.912201,c0,1,42111
4660644938PO2NC3A10C,1,0.680,3600000,Control,NaN,1,2023,PO,NaN,4.396003,c0,1,46606


In [9]:
import xarray as xr
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

def convert_mcds_to_anndata(mcds_path, cell_meta, cluster_col='L1', mc_type='CGN'):
    # Load the MCDS data and metadata
    mcds = MCDS.open(mcds_paths)

    
    # Filter metadata to ensure cells align with MCDS cells
    cell_meta = cell_metaa[cell_metaa.index.isin(mcds['cell'].values)]
    mcds = mcds.sel(cell=cell_meta.index)
    
    # Extract methylation fraction data directly from `genebody_da_frac`
    methylation_data = mcds['genebody_da_frac'].sel(mc_type=mc_type).values

    # Convert methylation data to a sparse matrix to save memory
    methylation_data = csr_matrix(methylation_data)

    # Create AnnData with methylation fraction matrix as `X`
    adata = anndata.AnnData(X=methylation_data)
    
    # Set cell-level metadata
    adata.obs = cell_meta
    
    # Set gene-level metadata
    adata.var = pd.DataFrame({
        'gene_id': mcds['genebody'].values,
        'chrom': mcds['genebody_chrom'].values,
        'start': mcds['genebody_start'].values,
        'end': mcds['genebody_end'].values
    }, index=mcds['genebody'].values)
    
    # Optionally, add the methylation fraction as a layer if needed for future analysis
    adata.layers['methylation_fraction'] = methylation_data

    return adata



In [10]:
#Convert MCDS to AnnData
adata = convert_mcds_to_anndata(mcds_paths, cell_meta, cluster_col='L1', mc_type='CGN')
adata

AnnData object with n_obs × n_vars = 46 × 26940
    obs: 'leiden', 'mCGFrac', 'TotalRead', 'Group', 'MT.Fraction', 'Collection', 'Year', 'Type', 'umap_0', 'umap_1', 'L1', 'L1_proba', 'AnimalID'
    var: 'gene_id', 'chrom', 'start', 'end'
    layers: 'methylation_fraction'

In [11]:
#
# Calculate DMGs
dmg_table = calculate_dmgs_scanpy(
    adata,
    cluster_col='L1',
    group_col='Group',
    method='wilcoxon',  # Pass as a string to avoid TypeError
    top_n=1000,
    pval_cutoff=0.05,
    adj_p_cutoff=0.1,
    logfc_min=0.8
)

dmg_table.to_hdf(f'{cluster_col}_DMG_CGN_ppoledoocytes.hdf', key='data') 
# View the results
print(dmg_table)


/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


   Cluster                Gene      pval   adjpval     logfc  \
0       c0  ENSMMUG00000004541  0.000315  0.668065  1.964730   
1       c0  ENSMMUG00000064061  0.000420  0.668065  0.916519   
2       c0  ENSMMUG00000047902  0.001365  0.668065  1.479827   
3       c0  ENSMMUG00000061125  0.001365  0.668065 -1.217649   
4       c0  ENSMMUG00000044556  0.001622  0.668065  1.044826   
..     ...                 ...       ...       ...       ...   
70      c0  ENSMMUG00000057382  0.046730  0.757343  0.961417   
71      c0  ENSMMUG00000013458  0.048128  0.757343  0.802934   
72      c0  ENSMMUG00000064772  0.048128  0.757343  1.708590   
73      c0  ENSMMUG00000064376  0.048128  0.757343  0.993567   
74      c0  ENSMMUG00000058428  0.049562  0.765037 -0.888083   

            Direction  
0   hyper methylation  
1   hyper methylation  
2   hyper methylation  
3    hypo methylation  
4   hyper methylation  
..                ...  
70  hyper methylation  
71  hyper methylation  
72  hyper methy

In [37]:
# Separate the data based on methylation direction

# 1. Separate hypo and hyper methylation entries
hypo_table = dmg_table[dmg_table['Direction'] == 'hypo methylation']
hyper_table = dmg_table[dmg_table['Direction'] == 'hyper methylation']

# 2. Merge with gene_meta to add gene coordinates
# (Assuming gene_meta has columns: 'Gene', 'chrom', 'start', 'end')
hypo_merged = pd.merge(hypo_table, gene_meta[['Gene', 'chrom', 'start', 'end']], on='Gene', how='left')
hyper_merged = pd.merge(hyper_table, gene_meta[['Gene', 'chrom', 'start', 'end']], on='Gene', how='left')
dmgall_merged = pd.merge(dmg_table, gene_meta[['Gene', 'chrom', 'start', 'end', 'gene_name']], on='Gene', how='left')

hypo_merged.to_csv('hypo_dmgs_mcg.csv', sep='\t')
hyper_merged.to_csv('hyper_dmgs_mcg.csv', sep='\t')
dmgall_merged.to_csv('all_dmgs_mcg.csv', sep='\t')
# 3. Prepare BED files: typically the BED format requires columns: chrom, start, end, and a name (here, using the Gene id)
# You can also include additional columns as needed.
hypo_bed = hypo_merged[['chrom', 'start', 'end']]
hyper_bed = hyper_merged[['chrom', 'start', 'end']]
all_bed = dmgall_merged[['chrom', 'start', 'end']]

# 4. Save to BED files (tab-delimited without header)
hypo_bed.to_csv('DMRs_hypo_mcg.bed', sep='\t', header=False, index=False)
hyper_bed.to_csv('DMRs_hyper_mcg.bed', sep='\t', header=False, index=False)
all_bed.to_csv('DMRs_mcg.bed',  sep='\t', header=False, index=False)

In [38]:
# background = gene_meta[['chrom', 'start', 'end']]
# background.to_csv("background.bed", sep='\t', header=False, index=False)

In [13]:
import xarray as xr
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

def convert_mcds_to_anndata(mcds_path, cell_meta, cluster_col='L1', mc_type='CHN'):
    # Load the MCDS data and metadata
    mcds = MCDS.open(mcds_paths)

    
    # Filter metadata to ensure cells align with MCDS cells
    cell_meta = cell_metaa[cell_metaa.index.isin(mcds['cell'].values)]
    mcds = mcds.sel(cell=cell_meta.index)
    
    # Extract methylation fraction data directly from `genebody_da_frac`
    methylation_data = mcds['genebody_da_frac'].sel(mc_type=mc_type).values

    # Convert methylation data to a sparse matrix to save memory
    methylation_data = csr_matrix(methylation_data)

    # Create AnnData with methylation fraction matrix as `X`
    adata = anndata.AnnData(X=methylation_data)
    
    # Set cell-level metadata
    adata.obs = cell_meta
    
    # Set gene-level metadata
    adata.var = pd.DataFrame({
        'gene_id': mcds['genebody'].values,
        'chrom': mcds['genebody_chrom'].values,
        'start': mcds['genebody_start'].values,
        'end': mcds['genebody_end'].values
    }, index=mcds['genebody'].values)
    
    # Optionally, add the methylation fraction as a layer if needed for future analysis
    adata.layers['methylation_fraction'] = methylation_data

    return adata



In [14]:
# Convert MCDS to AnnData
adata = convert_mcds_to_anndata(mcds_paths, cell_meta, cluster_col='L1', mc_type='CHN')
adata.write_h5ad(f'adata_genebody_pooled.h5ad')
# Calculate DMGs
dmg_table = calculate_dmgs_scanpy(
    adata,
    cluster_col='L1',
    group_col='Group',
    method='wilcoxon',  # Pass as a string to avoid TypeError
    top_n=1000,
    adj_p_cutoff=0.1,
    logfc_min=0.8
)

dmg_table.to_hdf(f'{cluster_col}_DMG_CHN_pooledoocytes.hdf', key='data') 
# View the results
print(dmg_table)


    Cluster                Gene      pval   adjpval     logfc  \
0        c0  ENSMMUG00000054173  0.000063  0.937666 -1.464418   
1        c0  ENSMMUG00000014197  0.000117  0.937666 -0.893818   
2        c0  ENSMMUG00000051652  0.000158  0.937666 -1.595667   
3        c0  ENSMMUG00000061041  0.000236  0.937666 -1.161796   
4        c0  ENSMMUG00000005377  0.000260  0.937666  1.172605   
..      ...                 ...       ...       ...       ...   
638      c0  ENSMMUG00000016539  0.048128  0.937666 -0.884620   
639      c0  ENSMMUG00000056787  0.048128  0.937666 -0.938700   
640      c0  ENSMMUG00000010331  0.048128  0.937666 -1.010697   
641      c0  ENSMMUG00000064224  0.049562  0.937666  1.014623   
642      c0  ENSMMUG00000040628  0.049562  0.937666  0.885886   

             Direction  
0     hypo methylation  
1     hypo methylation  
2     hypo methylation  
3     hypo methylation  
4    hyper methylation  
..                 ...  
638   hypo methylation  
639   hypo methylat

/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [18]:
# Separate the data based on methylation direction

# 1. Separate hypo and hyper methylation entries
hypo_table = dmg_table[dmg_table['Direction'] == 'hypo methylation']
hyper_table = dmg_table[dmg_table['Direction'] == 'hyper methylation']

# 2. Merge with gene_meta to add gene coordinates
# (Assuming gene_meta has columns: 'Gene', 'chrom', 'start', 'end')
hypo_merged = pd.merge(hypo_table, gene_meta[['Gene', 'chrom', 'start', 'end']], on='Gene', how='left')
hyper_merged = pd.merge(hyper_table, gene_meta[['Gene', 'chrom', 'start', 'end']], on='Gene', how='left')
dmgall_merged = pd.merge(dmg_table, gene_meta[['Gene', 'chrom', 'start', 'end', 'gene_name']], on='Gene', how='left')

hypo_merged.to_csv('hypo_dmgs_mch.csv', sep='\t')
hyper_merged.to_csv('hyper_dmgs_mch.csv', sep='\t')
dmgall_merged.to_csv('all_dmgs_mch.csv', sep='\t')
# 3. Prepare BED files: typically the BED format requires columns: chrom, start, end, and a name (here, using the Gene id)
# You can also include additional columns as needed.
hypo_bed = hypo_merged[['chrom', 'start', 'end']]
hyper_bed = hyper_merged[['chrom', 'start', 'end']]
all_bed = dmgall_merged[['chrom', 'start', 'end']]

# 4. Save to BED files (tab-delimited without header)
hypo_bed.to_csv('DMRs_hypo_mch.bed', sep='\t', header=False, index=False)
hyper_bed.to_csv('DMRs_hyper_mch.bed', sep='\t', header=False, index=False)
all_bed.to_csv('DMRs_mch.bed',  sep='\t', header=False, index=False)

In [17]:
# Convert MCDS to AnnData
adata = convert_mcds_to_anndata(mcds_paths, cell_meta, cluster_col='L1', mc_type='CHN')

# Run the updated function
dmg_table = calculate_dmgs_scanpy(adata, cluster_col='L1', group_col='Group', 
                                  method='wilcoxon', pval_cutoff=0.05, logfc_min=0.8)

dmg_table.to_hdf(f'{cluster_col}_DMG_CHN_pooledoocytes.hdf', key='data') 
# View the results
print(dmg_table)


    Cluster                Gene      pval   adjpval     logfc  \
0        c0  ENSMMUG00000054173  0.000063  0.937666 -1.464418   
1        c0  ENSMMUG00000014197  0.000117  0.937666 -0.893818   
2        c0  ENSMMUG00000051652  0.000158  0.937666 -1.595667   
3        c0  ENSMMUG00000061041  0.000236  0.937666 -1.161796   
4        c0  ENSMMUG00000005377  0.000260  0.937666  1.172605   
..      ...                 ...       ...       ...       ...   
638      c0  ENSMMUG00000016539  0.048128  0.937666 -0.884620   
639      c0  ENSMMUG00000056787  0.048128  0.937666 -0.938700   
640      c0  ENSMMUG00000010331  0.048128  0.937666 -1.010697   
641      c0  ENSMMUG00000064224  0.049562  0.937666  1.014623   
642      c0  ENSMMUG00000040628  0.049562  0.937666  0.885886   

             Direction  
0     hypo methylation  
1     hypo methylation  
2     hypo methylation  
3     hypo methylation  
4    hyper methylation  
..                 ...  
638   hypo methylation  
639   hypo methylat

/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [8]:
# calculate M-value for methylation 
import numpy as np
from scipy.sparse import csr_matrix

def convert_mcds_to_anndata(mcds_path, cell_meta, cluster_col='L1', mc_type='CGN'):
    # Load the MCDS data and metadata
    mcds = MCDS.open(mcds_path)  # corrected variable name if necessary
    
    # Filter metadata to ensure cells align with MCDS cells
    cell_meta = cell_meta[cell_meta.index.isin(mcds['cell'].values)]
    mcds = mcds.sel(cell=cell_meta.index)
    
    # Extract methylation fraction data directly from `genebody_da_frac`
    methylation_data = mcds['genebody_da_frac'].sel(mc_type=mc_type).values

    # Convert methylation data to a sparse matrix to save memory
    methylation_sparse = csr_matrix(methylation_data)

    # Create AnnData with methylation fraction matrix as `X`
    adata = anndata.AnnData(X=methylation_sparse)
    
    # Set cell-level metadata
    adata.obs = cell_meta
    
    # Set gene-level metadata
    adata.var = pd.DataFrame({
        'gene_id': mcds['genebody'].values,
        'chrom': mcds['genebody_chrom'].values,
        'start': mcds['genebody_start'].values,
        'end': mcds['genebody_end'].values
    }, index=mcds['genebody'].values)
    
    # Optionally, add the methylation fraction as a layer if needed for future analysis
    adata.layers['methylation_fraction'] = methylation_sparse

    # Convert beta values to M-values
    # First, convert sparse matrix to dense (if memory allows) for the transformation.
    methylation_dense = methylation_sparse.toarray()
    epsilon = 1e-5  # Small constant to avoid log(0)
    # Clip the beta values to avoid 0 or 1
    beta_clipped = np.clip(methylation_dense, epsilon, 1 - epsilon)
    # Compute M-values
    M_values = np.log2(beta_clipped / (1 - beta_clipped))
    # Optionally, convert back to a sparse matrix to save memory
    adata.layers['M_values'] = csr_matrix(M_values)

    return adata


In [9]:
#Convert MCDS to AnnData
adata = convert_mcds_to_anndata(mcds_paths, cell_meta, cluster_col='L1', mc_type='CGN')
adata

AnnData object with n_obs × n_vars = 46 × 26940
    obs: 'leiden', 'mCGFrac', 'TotalRead', 'Group', 'MT.Fraction', 'Collection', 'Year', 'Type', 'umap_0', 'umap_1', 'L1', 'L1_proba', 'AnimalID'
    var: 'gene_id', 'chrom', 'start', 'end'
    layers: 'methylation_fraction', 'M_values'

In [19]:
#
# Calculate DMGs
dmg_table = calculate_dmgs_pvaladj_scanpy(
    adata,
    cluster_col='L1',
    group_col='Group',
    method='wilcoxon',  # Pass as a string to avoid TypeError
    top_n=1000,
    pval_cutoff=0.05,
    logfc_min=0.8
)

dmg_table.to_hdf(f'{cluster_col}_DMG_CGN_pooledoocytes-mvalue.hdf', key='data') 
# View the results
print(dmg_table)


/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/ehabibi/miniconda3/envs/allcools/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


    Cluster                Gene      pval  adjpval      logfc
0        c0  ENSMMUG00000020502  0.000056      1.0 -13.667381
1        c0  ENSMMUG00000058972  0.000143      1.0  10.485583
2        c0  ENSMMUG00000017711  0.000158      1.0  10.825211
3        c0  ENSMMUG00000064061  0.000286      1.0 -16.014212
4        c0  ENSMMUG00000017600  0.000315      1.0   8.441697
..      ...                 ...       ...      ...        ...
673      c0  ENSMMUG00000020880  0.049562      1.0  -7.100075
674      c0  ENSMMUG00000015654  0.049562      1.0  -1.804622
675      c0  ENSMMUG00000023691  0.049562      1.0  -8.201190
676      c0  ENSMMUG00000015522  0.049562      1.0  -6.090725
677      c0  ENSMMUG00000054896  0.049562      1.0  -9.575774

[678 rows x 5 columns]
